In [1]:
import panel as pn

In [2]:
height_widgets = 50

In [3]:
button_login = pn.widgets.Button(name='ENTRAR', button_type='primary', height=height_widgets)
button_registration = pn.widgets.Button(name='Registre-se', height=height_widgets)

button_registration_client = pn.widgets.Button(name='Cadastrar', button_type='primary', height=height_widgets, width=620)
button_registration_company = pn.widgets.Button(name='Cadastrar', button_type='primary', height=height_widgets, width=620)
button_registration_unsubscribe = pn.widgets.Button(name='Cancelar cadastro', button_type='success', height=height_widgets, width=620)

In [4]:
layout_main = pn.Column()

In [5]:
template = pn.template.FastListTemplate(
    site="Fornecer",
    title="Dynamic Plot Layout",
    main=[layout_main],
    main_max_width='385px'
)

In [6]:
def start():
    screen_login('start')

def screen_login(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    template.main_max_width = '385px'
    
    layout_main.append(pn.Row('<h2>Faça seu login</h2>'))
    layout_main.append(pn.Row(pn.widgets.TextInput(placeholder='Cpf/Cnpj', height=height_widgets)))
    layout_main.append(pn.Row(pn.widgets.PasswordInput(placeholder='Senha', height=height_widgets)))
    layout_main.append(pn.Row(button_login))
    layout_main.append(pn.Row('Ainda não tem uma conta?'))
    layout_main.append(pn.Row(button_registration))

def screen_register(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    template.main_max_width = '700px'

    register_client = pn.Column(
        pn.Row('<h3>Dados do cliente</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cpf', height=height_widgets), pn.widgets.TextInput(placeholder='Email', height=height_widgets)),
        pn.Row(pn.widgets.TextInput(placeholder='Nome', height=height_widgets), pn.widgets.TextInput(placeholder='Sobrenome', height=height_widgets)),
        pn.Row('<h3>Localização</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cep', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Região', height=height_widgets))),
        pn.Row(pn.widgets.TextInput(placeholder='Estado', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Cidade', height=height_widgets))),
        pn.Row(button_registration_client)
    )

    register_company = pn.Column(
        pn.Row('<h3>Dados da empresa</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cnpj', height=height_widgets), pn.widgets.TextInput(placeholder='Email', height=height_widgets)),
        pn.Row(pn.widgets.TextInput(placeholder='Nome da empresa', height=height_widgets), pn.widgets.TextInput(placeholder='Sobrenome', height=height_widgets)),
        pn.Row(pn.widgets.TextInput(placeholder='Mercado de atuação', height=height_widgets)),
        pn.Row('<h3>Localização</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cep', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Região', height=height_widgets))),
        pn.Row(pn.widgets.TextInput(placeholder='Estado', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Cidade', height=height_widgets))),
        pn.Row(button_registration_company)
    )

    tab_registration = pn.Tabs(
        ('Cliente', register_client),
        ('Empresa', register_company)
    )
    
    layout_main.append(pn.Row('<h2>Crie sua conta</h2>'))
    layout_main.append(tab_registration)
    layout_main.append(button_registration_unsubscribe)

In [7]:
button_registration.param.watch(screen_register, 'value')
button_registration.value = False

button_registration_unsubscribe.param.watch(screen_login, 'value')
button_registration_unsubscribe.value = False

In [8]:
start()

template.show()

Launching server at http://localhost:57358
